In [1]:
from nupack import Domain, TargetStrand, TargetComplex, TargetTube, SetSpec, Model, tube_design

In [3]:
hcj = Domain('N', name='hcj');
hck = Domain('N', name='hck');
hbr = Domain('N', name='hbr');
sb = Domain('N', name='sb');
sc = Domain('N', name='sc');
fc = Domain('N', name='fc');
fb = Domain('N', name='fb');
mb = Domain('N', name='mb');
mc = Domain('N', name='mc');


In [4]:
#react step species
Cj = TargetStrand([sc, mc, fc, hcj], name='Cj');
BackCB = TargetStrand([fb,sc,mc], name = 'BackCB');
FluxBCj = TargetStrand([hcj, sb, mb], name='FluxBCj');
sweeplineReact = TargetStrand([~fc,~mc,~sc,~fb,~mb,~sb], name = 'sweeplineReact');
Br = TargetStrand([sb, mb, fb, hbr], name = 'Br');

# produce step species
Ck = TargetStrand([sc,mc,fc,hck], name = 'Ck');
sweeplineProduce = TargetStrand([~sb, ~hcj, ~fc, ~hck, ~fc], name = 'sweeplineProduce');
HelperCCk = TargetStrand([fc,hck,fc], name = 'HelperCCk');

In [6]:
#react step complexes
ReactIntCBCj = TargetComplex([Cj, sweeplineReact, FluxBCj ], "(((.+))).((+.))", name = 'ReactIntCBCj')
ReactCBCj = TargetComplex([BackCB, sweeplineReact, FluxBCj], "(((+.)))((+.))", name = 'ReactCBCj')
WasteCjBr = TargetComplex([Cj, sweeplineReact, Br], "(((.+)))(((+))).", name = 'WasteCjBr');

#produce step complexes
ProduceBCjCk = TargetComplex([Cj, sweeplineProduce, Ck], "..((+.))((+..))", name = 'ProduceBCjCk');
ProduceIntBCjCk = TargetComplex([FluxBCj, sweeplineProduce, Ck], "((.+)).((+..))", name = 'ProducetIntBCjCk');
WasteBCjCk = TargetComplex([FluxBCj, sweeplineProduce, HelperCCk], "((.+))(((+)))", name = 'WasteBCjCk');

In [8]:
#leak complexes
LeakedReactCBr = TargetComplex([BackCB, sweeplineReact, Br], "(((+.)))((+))..", name = 'LeakedReactCBr');
LeakedProduceBCjCk = TargetComplex([Cj, sweeplineProduce, HelperCCk], "...(+.)(((+)))", name = 'LeakedProduceBCjCk');
# LeakedReactCBCjProduceBCjCk = TargetComplex([BackCB, sweeplineReact, sweeplineProduce, Ck, FluxBCj], "(((+((+(+.)))).+)).((+..))", name = 'LeakedReactCBCjProduceBCjCk');

In [10]:
autocatalytic_amplifier_tube = TargetTube(on_targets={WasteBCjCk: 1e-8, WasteCjBr: 1e-8, ReactCBCj: 1e-8, ReactIntCBCj: 1e-8, ProduceBCjCk: 1e-8, ProduceIntBCjCk: 1e-8}, name='t1',
    off_targets=SetSpec(max_size=3, include=[LeakedReactCBr, LeakedProduceBCjCk], exclude=[ReactCBCj, ReactIntCBCj, ProduceBCjCk, ProduceIntBCjCk]))

sequence_design_model = Model();
reaction_tubes = [autocatalytic_amplifier_tube];
design = tube_design(tubes=reaction_tubes, model=sequence_design_model)

In [12]:
results = design.run(trials=3)
results[0]

Domain,Sequence
fb,G
fb*,C
fc,C
fc*,G
hbr,G
hbr*,C
hcj,G
hcj*,C
hck,C
hck*,G
